# March Madness
## Final Project
### Group 15: Andrew Marion, Dallas Hutchinson, Aydan Koyles


# Setting Up File

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Imports

In [6]:
import json
from pyspark.mllib.classification import LogisticRegressionModel,LogisticRegressionWithLBFGS, SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.clustering import *
import pyspark.sql
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import LDA
import numpy as np
import pyspark.sql.functions as f

# Read in csv file

In [7]:
RegularSeason = spark.read.option("header",True).csv("/content/gdrive/MyDrive/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv", inferSchema = True)
Seeds = spark.read.option("header",True).csv("/content/gdrive/MyDrive/MDataFiles_Stage1/MNCAATourneySeeds.csv", inferSchema = True)
Tourney = spark.read.option("header",True).csv("/content/gdrive/MyDrive/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv", inferSchema = True)
Massey = spark.read.option("header",True).csv("/content/gdrive/MyDrive/MDataFiles_Stage1/MMasseyOrdinals.csv", inferSchema = True)
Conferences = spark.read.option("header",True).csv("/content/gdrive/MyDrive/MDataFiles_Stage1/MNCAATourneySeeds.csv", inferSchema = True)
Coaches = spark.read.option("header",True).csv("/content/gdrive/MyDrive/MDataFiles_Stage1/MTeamCoaches.csv", inferSchema = True)

In [ ]:
Coaches.show()

+------+------+-----------+----------+---------------+
|Season|TeamID|FirstDayNum|LastDayNum|      CoachName|
+------+------+-----------+----------+---------------+
|  1985|  1102|          0|       154|  reggie_minton|
|  1985|  1103|          0|       154|    bob_huggins|
|  1985|  1104|          0|       154| wimp_sanderson|
|  1985|  1106|          0|       154|   james_oliver|
|  1985|  1108|          0|       154|  davey_whitney|
|  1985|  1109|          0|       154|   freddie_goss|
|  1985|  1110|          0|       154|    ed_tapscott|
|  1985|  1111|          0|       154| kevin_cantwell|
|  1985|  1112|          0|       154|     lute_olson|
|  1985|  1113|          0|       154|  bob_weinhauer|
|  1985|  1114|          0|       154|    mike_newell|
|  1985|  1116|          0|       154|   eddie_sutton|
|  1985|  1117|          0|       154|nelson_catalina|
|  1985|  1119|          0|       154|     les_wothke|
|  1985|  1120|          0|       154|    sonny_smith|
|  1985|  

# Clean / Explore Data

In [24]:
WinTeams = RegularSeason.groupBy('Season', 'WTeamID').sum('WScore','LScore','NumOT','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA','WOR','WDR','WAst','WTO','WStl','WBlk','WPF','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA','LOR','LDR','LAst','LTO','LStl','LBlk','LPF')
LoseTeams = RegularSeason.groupBy('Season', 'LTeamID').sum('WScore','LScore','NumOT','WFGM','WFGA','WFGM3','WFGA3','WFTM','WFTA','WOR','WDR','WAst','WTO','WStl','WBlk','WPF','LFGM','LFGA','LFGM3','LFGA3','LFTM','LFTA','LOR','LDR','LAst','LTO','LStl','LBlk','LPF')

In [ ]:
for column in [column for column in dataframe2.columns
               if column not in dataframe1.columns]:
    dataframe1 = dataframe1.withColumn(column, lit(None))

NameError: ignored

In [31]:
NumWin = RegularSeason.groupBy('Season', 'WTeamID').count()
NumWin = NumWin.withColumnRenamed("count", "Wins")
NumWin = NumWin.withColumnRenamed("WTeamID", "TeamID")

NumLose = RegularSeason.groupBy('Season', 'LTeamID').count()
NumLose = NumLose.withColumnRenamed("count", "Loses")
NumLose = NumLose.withColumnRenamed("LTeamID", "TeamID")

merged_df = NumWin.join(NumLose, ['Season','TeamID'])
merged_df = merged_df.withColumn('NumGames', merged_df.Wins + merged_df.Loses)
merged_df.show()

+------+------+----+-----+--------+
|Season|TeamID|Wins|Loses|NumGames|
+------+------+----+-----+--------+
|  2003|  1226|  13|   14|      27|
|  2003|  1189|  10|   15|      25|
|  2009|  1401|  23|    9|      32|
|  2011|  1431|  24|    9|      33|
|  2006|  1173|  13|   17|      30|
|  2006|  1153|  18|   12|      30|
|  2007|  1275|  18|   14|      32|
|  2007|  1175|  20|   12|      32|
|  2008|  1145|   6|   25|      31|
|  2008|  1313|  16|   15|      31|
|  2008|  1229|  23|    9|      32|
|  2008|  1325|  19|   12|      31|
|  2009|  1286|  13|   17|      30|
|  2009|  1189|   9|   20|      29|
|  2009|  1267|  12|   17|      29|
|  2012|  1334|   9|   19|      28|
|  2013|  1158|  24|    9|      33|
|  2013|  1104|  20|   12|      32|
|  2015|  1165|  12|   17|      29|
|  2015|  1337|  17|   13|      30|
+------+------+----+-----+--------+
only showing top 20 rows



for win team: rename all w to generaric ones and all L to opp as they are opponent stats.

Also add count to get number of wins




for lose team: rename all L to generaric ones and all W to opp as they are opponent stats

Also add count to get number of loses

the combine them off of team id

In [28]:
WinTeams.show()
LoseTeams.show()

+------+-------+-----------+-----------+----------+---------+---------+----------+----------+---------+---------+--------+--------+---------+--------+---------+---------+--------+---------+---------+----------+----------+---------+---------+--------+--------+---------+--------+---------+---------+--------+
|Season|WTeamID|sum(WScore)|sum(LScore)|sum(NumOT)|sum(WFGM)|sum(WFGA)|sum(WFGM3)|sum(WFGA3)|sum(WFTM)|sum(WFTA)|sum(WOR)|sum(WDR)|sum(WAst)|sum(WTO)|sum(WStl)|sum(WBlk)|sum(WPF)|sum(LFGM)|sum(LFGA)|sum(LFGM3)|sum(LFGA3)|sum(LFTM)|sum(LFTA)|sum(LOR)|sum(LDR)|sum(LAst)|sum(LTO)|sum(LStl)|sum(LBlk)|sum(LPF)|
+------+-------+-----------+-----------+----------+---------+---------+----------+----------+---------+---------+--------+--------+---------+--------+---------+---------+--------+---------+---------+----------+----------+---------+---------+--------+--------+---------+--------+---------+---------+--------+
|  2003|   1226|        974|        880|         2|      342|      729|     

In [29]:
WinTeams = WinTeams.withColumnRenamed("WTeamID", "TeamID")
WinTeams = WinTeams.withColumnRenamed("sum(WScore)", "PTS")
WinTeams = WinTeams.withColumnRenamed("sum(LScore)", "OppPTS")
WinTeams = WinTeams.withColumnRenamed("sum(NumOT)", "OTs")
WinTeams = WinTeams.withColumnRenamed("sum(WFGM)", "FGM")
WinTeams = WinTeams.withColumnRenamed("sum(WFGA)", "FGA")
WinTeams = WinTeams.withColumnRenamed("sum(WFGA3)", "FGA3")
WinTeams = WinTeams.withColumnRenamed("sum(WFGM3)", "FGM3")
WinTeams = WinTeams.withColumnRenamed("sum(WFTA)", "FTA")
WinTeams = WinTeams.withColumnRenamed("sum(WFTM)", "FTM")
WinTeams = WinTeams.withColumnRenamed("sum(WOR)", "OR")
WinTeams = WinTeams.withColumnRenamed("sum(WDR)", "DR")
WinTeams = WinTeams.withColumnRenamed("sum(WAst)", "AST")
WinTeams = WinTeams.withColumnRenamed("sum(WTO)", "TO")
WinTeams = WinTeams.withColumnRenamed("sum(WStl)", "STL")
WinTeams = WinTeams.withColumnRenamed("sum(WBlk)", "BLK")
WinTeams = WinTeams.withColumnRenamed("sum(WPF)", "PF")
WinTeams = WinTeams.withColumnRenamed("sum(LFGM)", "OppFGM")
WinTeams = WinTeams.withColumnRenamed("sum(LFGA)", "OppFGA")
WinTeams = WinTeams.withColumnRenamed("sum(LFGA3)", "OppFGA3")
WinTeams = WinTeams.withColumnRenamed("sum(LFGM3)", "OppFGM3")
WinTeams = WinTeams.withColumnRenamed("sum(LFTA)", "OppFTA")
WinTeams = WinTeams.withColumnRenamed("sum(LFTM)", "OppFTM")
WinTeams = WinTeams.withColumnRenamed("sum(LOR)", "OppOR")
WinTeams = WinTeams.withColumnRenamed("sum(LDR)", "OppDR")
WinTeams = WinTeams.withColumnRenamed("sum(LAst)", "OppAST")
WinTeams = WinTeams.withColumnRenamed("sum(LTO)", "OppTO")
WinTeams = WinTeams.withColumnRenamed("sum(LStl)", "OppSTL")
WinTeams = WinTeams.withColumnRenamed("sum(LBlk)", "OppBLK")
WinTeams = WinTeams.withColumnRenamed("sum(LPF)", "OppPF")
WinTeams.show()

+------+------+----+------+---+---+----+----+----+---+---+---+---+---+---+---+---+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+
|Season|TeamID| PTS|OppPTS|OTs|FGM| FGA|FGM3|FGA3|FTM|FTA| OR| DR|AST| TO|STL|BLK| PF|OppFGM|OppFGA|OppFGM3|OppFGA3|OppFTM|OppFTA|OppOR|OppDR|OppAST|OppTO|OppSTL|OppBLK|OppPF|
+------+------+----+------+---+---+----+----+----+---+---+---+---+---+---+---+---+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+
|  2003|  1226| 974|   880|  2|342| 729|  86| 218|204|312|136|326|217|192|110| 57|237|   306|   752|     80|    244|   188|   264|  153|  294|   178|  203|    87|    33|  279|
|  2003|  1189| 672|   590|  0|244| 511|  42| 119|142|193|119|234|112|158| 75| 26|161|   223|   527|     53|    165|    91|   148|  100|  175|   100|  135|    70|    32|  167|
|  2009|  1401|1722|  1473|  1|579|1243| 146| 375|418|587|285|584|314|275|124| 92|366|   551|  1307|    149|    417|   2

In [30]:
LoseTeams = LoseTeams.withColumnRenamed("LTeamID", "TeamID")
LoseTeams = LoseTeams.withColumnRenamed("sum(WScore)", "OppPTS")
LoseTeams = LoseTeams.withColumnRenamed("sum(LScore)", "PTS")
LoseTeams = LoseTeams.withColumnRenamed("sum(NumOT)", "OTs")
LoseTeams = LoseTeams.withColumnRenamed("sum(WFGM)", "OppFGM")
LoseTeams = LoseTeams.withColumnRenamed("sum(WFGA)", "OppFGA")
LoseTeams = LoseTeams.withColumnRenamed("sum(WFGA3)", "OppFGA3")
LoseTeams = LoseTeams.withColumnRenamed("sum(WFGM3)", "OppFGM3")
LoseTeams = LoseTeams.withColumnRenamed("sum(WFTA)", "OppFTA")
LoseTeams = LoseTeams.withColumnRenamed("sum(WFTM)", "OppFTM")
LoseTeams = LoseTeams.withColumnRenamed("sum(WOR)", "OppOR")
LoseTeams = LoseTeams.withColumnRenamed("sum(WDR)", "OppDR")
LoseTeams = LoseTeams.withColumnRenamed("sum(WAst)", "OppAST")
LoseTeams = LoseTeams.withColumnRenamed("sum(WTO)", "OppTO")
LoseTeams = LoseTeams.withColumnRenamed("sum(WStl)", "OppSTL")
LoseTeams = LoseTeams.withColumnRenamed("sum(WBlk)", "OppBLK")
LoseTeams = LoseTeams.withColumnRenamed("sum(WPF)", "OppPF")
LoseTeams = LoseTeams.withColumnRenamed("sum(LFGM)", "FGM")
LoseTeams = LoseTeams.withColumnRenamed("sum(LFGA)", "FGA")
LoseTeams = LoseTeams.withColumnRenamed("sum(LFGA3)", "FGA3")
LoseTeams = LoseTeams.withColumnRenamed("sum(LFGM3)", "FGM3")
LoseTeams = LoseTeams.withColumnRenamed("sum(LFTA)", "FTA")
LoseTeams = LoseTeams.withColumnRenamed("sum(LFTM)", "FTM")
LoseTeams = LoseTeams.withColumnRenamed("sum(LOR)", "OR")
LoseTeams = LoseTeams.withColumnRenamed("sum(LDR)", "DR")
LoseTeams = LoseTeams.withColumnRenamed("sum(LAst)", "AST")
LoseTeams = LoseTeams.withColumnRenamed("sum(LTO)", "TO")
LoseTeams = LoseTeams.withColumnRenamed("sum(LStl)", "STL")
LoseTeams = LoseTeams.withColumnRenamed("sum(LBlk)", "BLK")
LoseTeams = LoseTeams.withColumnRenamed("sum(LPF)", "PF")
LoseTeams.show()

+------+------+------+----+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+---+----+----+----+---+---+---+---+---+---+---+---+---+
|Season|TeamID|OppPTS| PTS|OTs|OppFGM|OppFGA|OppFGM3|OppFGA3|OppFTM|OppFTA|OppOR|OppDR|OppAST|OppTO|OppSTL|OppBLK|OppPF|FGM| FGA|FGM3|FGA3|FTM|FTA| OR| DR|AST| TO|STL|BLK| PF|
+------+------+------+----+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+---+----+----+----+---+---+---+---+---+---+---+---+---+
|  2003|  1226|  1042| 932|  2|   345|   738|     95|    252|   257|   362|  168|  367|   204|  248|    96|    44|  241|356| 846|  62| 219|158|250|168|283|168|212|107| 35|300|
|  2003|  1189|  1121| 888|  3|   360|   748|    100|    259|   301|   381|  154|  361|   201|  216|   139|    68|  261|329| 844|  71| 230|159|264|204|280|167|247|105| 38|326|
|  2009|  1401|   650| 573|  0|   227|   497|     52|    138|   144|   196|   98|  213|   108|  111|    61|    32|  187|

In [37]:
LoseTeams.filter(LoseTeams.Season == 2022).show()

+------+------+------+----+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+---+----+----+----+---+---+---+---+---+---+---+---+---+
|Season|TeamID|OppPTS| PTS|OTs|OppFGM|OppFGA|OppFGM3|OppFGA3|OppFTM|OppFTA|OppOR|OppDR|OppAST|OppTO|OppSTL|OppBLK|OppPF|FGM| FGA|FGM3|FGA3|FTM|FTA| OR| DR|AST| TO|STL|BLK| PF|
+------+------+------+----+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+---+----+----+----+---+---+---+---+---+---+---+---+---+
|  2022|  1175|  1228| 913|  0|   424|   985|    168|    450|   212|   295|  184|  395|   278|  199|   130|    57|  288|326| 863|  76| 268|185|279|132|345|130|260| 95| 70|273|
|  2022|  1180|   762| 691|  2|   284|   585|     69|    182|   125|   172|   67|  226|   120|  101|    87|    24|  152|262| 594|  73| 227| 94|124| 87|223|110|135| 38| 31|161|
|  2022|  1153|   513| 445|  0|   175|   408|     46|    125|   117|   160|   77|  189|    93|   76|    37|    21|  124|

In [33]:
merge = WinTeams.join(LoseTeams, ['Season','TeamID'])
merge.show()

+------+------+----+------+---+---+----+----+----+---+---+---+---+---+---+---+---+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+------+----+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+---+----+----+----+---+---+---+---+---+---+---+---+---+
|Season|TeamID| PTS|OppPTS|OTs|FGM| FGA|FGM3|FGA3|FTM|FTA| OR| DR|AST| TO|STL|BLK| PF|OppFGM|OppFGA|OppFGM3|OppFGA3|OppFTM|OppFTA|OppOR|OppDR|OppAST|OppTO|OppSTL|OppBLK|OppPF|OppPTS| PTS|OTs|OppFGM|OppFGA|OppFGM3|OppFGA3|OppFTM|OppFTA|OppOR|OppDR|OppAST|OppTO|OppSTL|OppBLK|OppPF|FGM| FGA|FGM3|FGA3|FTM|FTA| OR| DR|AST| TO|STL|BLK| PF|
+------+------+----+------+---+---+----+----+----+---+---+---+---+---+---+---+---+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+------+----+---+------+------+-------+-------+------+------+-----+-----+------+-----+------+------+-----+---+----+----+----+---+---+---+---+---+---+---+-

TO DO

1.   Combine all regular season stats
2.   Break off 2022
3.   Get 2022 seeds
4.   Make Model

